In [141]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV


In [142]:
pd.set_option('display.max_column',None)

In [143]:
#load data
dataset = pd.read_csv("finaldatav10.csv")
dataset.head(1)


,url,title,location,rating,type_airbnb,gbbb,guests,Numberofbeds,Numberofbedrooms,Numberofbathrooms,price per night,host_tags,host_response,amenities,latitude,longitude,Cleanliness,Accuracy,Communication,Location_rating,Check-in,Value,desc_tags,numlistingsprovince,population,area,travelers2019,foreigntravelers,covid_total,covid_active,superhost,Host_IdentityVerified,waterfront,compactor,securitycameras,sauna,parking,washingmachine,gameconsole,gym,balcony,piano,swimmingpool,indoorgames,wifi,hairdryer,hangers,bathtub,hotwater,iron,karaoke,safe,keypad,kitchen,kettle,tv,shampooconditioner,essentials,refrigerator,streamingservice,airconditioning,soundsystem,oven,grill,babyfriendly,utensils,linen,bidet,breakfast,staff,alarm,fan,heating,childfriendly,clothesstorage,coffeemaker,dedicatedworkspace,diningtable,dryer,cookingbasics,elevator,beddings,garden,fireextinguisher,firepit,fireplaceguards,firstadkit,entirehome,enhancedclean,cancellationpolicy,experiencedhost,freeparkingonpremises,greatcheckinexperience,greatcommunication,greatlocation,highlyratedhost,houserules,laptopfriendlyworkspace,outstandinghospitality,petsallowed,selfcheckin,parklingclean,issharedroom,isprivateroom,numnearbylistings,numneartouristspots3km,numneartouristspots5km,numnearbymall5km,numnearbysupermarket3km,numnearbysupermarket5km,numnearbyrestaurants3km,numnearbyrestaurants5km,numnearpublictranspo,distance_airport,province,Abra,Agusan del Norte,Agusan del Sur,Aklan,Albay,Antique,Apayao,Aurora,Bataan,Batanes,Batangas,Benguet,Biliran,Bohol,Bukidnon,Bulacan,Cagayan,Caloocan City,Camarines Norte,Camarines Sur,Camiguin,Capiz,Catanduanes,Cavite,Cebu,Compostela Valley,Cotabato,Davao Occidental,Davao Oriental,Davao del Norte,Davao del Sur,Dinagat Islands,Eastern Samar,Guimaras,Ifugao,Ilocos Norte,Ilocos Sur,Iloilo,Isabela,Kalinga,La Union,Laguna,Lanao del Norte,Lanao del Sur,Las Piñas City,Leyte,Maguindanao,Makati City,Malabon City,Mandaluyong City,Manila City,Marikina City,Marinduque,Masbate,Misamis Occidental,Misamis Oriental,Mountain Province,Muntinlupa City,Navotas City,Negros Occidental,Negros Oriental,Northern Samar,Nueva Ecija,Nueva Vizcaya,Occidental Mindoro,Oriental Mindoro,Palawan,Pampanga,Pangasinan,Parañaque City,Pasay City,Pasig City,Pateros,Quezon,Quezon City,Quirino,Rizal,Romblon,Samar,San Juan City,Sarangani,Siquijor,Sorsogon,South Cotabato,Southern Leyte,Sultan Kudarat,Surigao del Norte,Surigao del Sur,Taguig City,Tarlac,Valenzuela City,Zambales,Zamboanga Sibugay,Zamboanga del Norte,Zamboanga del Sur,num_reviews,within3km,coast750m,type_airbnb_label,province_label,private_room,private_room_in_cave,shared_room,houseboat,campsite,earth_house,entire_place,entire_resort,entire_villa,farm_stay,island,domestictravels
0,https://www.airbnb.com/rooms/39300057?previous...,Hidden Valley Sunset Stonehouse,"Lazi, Central Visayas, Philippines",NaN,hut,"['2 guests ', ' 1 bedroom ', ' 0 beds ', ' 1 b...",2,0,1,1.0,1900,"['14 reviews', 'superhost']",[],"['shampoo', 'hot water', 'essentials', 'hanger...",9.13261,123.6001,NaN,NaN,NaN,NaN,NaN,NaN,"['entire home', 'enhanced clean', 'richard is ...",220,95984,337,168366,85713,89,14,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,2,5,0,0,0,0,2,0,39.603528,Siquijor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,7,82,0,0,0,0,0,0,1,0,0,0,0,82653


In [144]:
Y = dataset['price per night']
Y

0         1900
1          850
2          506
3         1400
4          750
5         1000
6         1500
7          850
8          850
9         1150
10        1000
11         900
12         506
13        1200
14         567
15         900
16        4865
17        1100
18        1000
19         980
20        3000
21        7000
22        3000
23         820
24        1200
25         850
26        1050
27         750
28        1500
29        1800
30        2500
31         800
32         508
33         800
34        1700
35        5857
36        1700
37        3750
38        1600
39        1300
40         750
41         730
42        2500
43        3000
44        1200
45        1000
46         730
47        2250
48        1150
49        3400
50        1300
51        6700
52        1100
53        1362
54        1250
55        1950
56         730
57        1800
58        1650
59        1500
60        1800
61        4000
62        3741
63        1200
64        1100
65        1250
66        

In [145]:
# the independent variables set
X = dataset.drop(['price per night'],axis=1)

In [146]:
X = X.drop(columns=['url','title','location','rating','type_airbnb','gbbb',\
                    'host_tags', 'host_response', 'amenities', 'desc_tags',\
                    'province', 'issharedroom', 'isprivateroom', 'numneartouristspots3km',\
                    'numnearbysupermarket3km', 'numnearbysupermarket5km','numnearbyrestaurants3km','entirehome',\
                    'numlistingsprovince', 'latitude', 'longitude',\
                    'population','area','travelers2019','foreigntravelers','covid_total','covid_active',\
                    'Cleanliness','Accuracy','Communication','Location_rating','Check-in','Value'])

In [147]:
Prov = X.iloc[:,81:176].columns
Prov

Index(['Abra', 'Agusan del Norte', 'Agusan del Sur', 'Aklan', 'Albay',
       'Antique', 'Apayao', 'Aurora', 'Bataan', 'Batanes', 'Batangas',
       'Benguet', 'Biliran', 'Bohol', 'Bukidnon', 'Bulacan', 'Cagayan',
       'Caloocan City', 'Camarines Norte', 'Camarines Sur', 'Camiguin',
       'Capiz', 'Catanduanes', 'Cavite', 'Cebu', 'Compostela Valley',
       'Cotabato', 'Davao Occidental', 'Davao Oriental', 'Davao del Norte',
       'Davao del Sur', 'Dinagat Islands', 'Eastern Samar', 'Guimaras',
       'Ifugao', 'Ilocos Norte', 'Ilocos Sur', 'Iloilo', 'Isabela', 'Kalinga',
       'La Union', 'Laguna', 'Lanao del Norte', 'Lanao del Sur',
       'Las Piñas City', 'Leyte', 'Maguindanao', 'Makati City', 'Malabon City',
       'Mandaluyong City', 'Manila City', 'Marikina City', 'Marinduque',
       'Masbate', 'Misamis Occidental', 'Misamis Oriental',
       'Mountain Province', 'Muntinlupa City', 'Navotas City',
       'Negros Occidental', 'Negros Oriental', 'Northern Samar', 'Nueva Ecij

In [148]:
Type = X.iloc[:,181:].columns
Type

Index(['private_room', 'private_room_in_cave', 'shared_room', 'houseboat',
       'campsite', 'earth_house', 'entire_place', 'entire_resort',
       'entire_villa', 'farm_stay', 'island', 'domestictravels'],
      dtype='object')

In [149]:
X =X.drop(columns=Type)

In [150]:
X =X.drop(columns=Prov)

In [151]:
X.head(1)

,guests,Numberofbeds,Numberofbedrooms,Numberofbathrooms,superhost,Host_IdentityVerified,waterfront,compactor,securitycameras,sauna,parking,washingmachine,gameconsole,gym,balcony,piano,swimmingpool,indoorgames,wifi,hairdryer,hangers,bathtub,hotwater,iron,karaoke,safe,keypad,kitchen,kettle,tv,shampooconditioner,essentials,refrigerator,streamingservice,airconditioning,soundsystem,oven,grill,babyfriendly,utensils,linen,bidet,breakfast,staff,alarm,fan,heating,childfriendly,clothesstorage,coffeemaker,dedicatedworkspace,diningtable,dryer,cookingbasics,elevator,beddings,garden,fireextinguisher,firepit,fireplaceguards,firstadkit,enhancedclean,cancellationpolicy,experiencedhost,freeparkingonpremises,greatcheckinexperience,greatcommunication,greatlocation,highlyratedhost,houserules,laptopfriendlyworkspace,outstandinghospitality,petsallowed,selfcheckin,parklingclean,numnearbylistings,numneartouristspots5km,numnearbymall5km,numnearbyrestaurants5km,numnearpublictranspo,distance_airport,num_reviews,within3km,coast750m,type_airbnb_label,province_label
0,2,0,1,1.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,5,0,2,0,39.603528,1,0,0,7,82


In [152]:
pd.set_option('display.max_row',None)

In [153]:
X.dtypes

guests                       int64
Numberofbeds                 int64
Numberofbedrooms             int64
Numberofbathrooms          float64
superhost                    int64
Host_IdentityVerified        int64
waterfront                   int64
compactor                    int64
securitycameras              int64
sauna                        int64
parking                      int64
washingmachine               int64
gameconsole                  int64
gym                          int64
balcony                      int64
piano                        int64
swimmingpool                 int64
indoorgames                  int64
wifi                         int64
hairdryer                    int64
hangers                      int64
bathtub                      int64
hotwater                     int64
iron                         int64
karaoke                      int64
safe                         int64
keypad                       int64
kitchen                      int64
kettle              

In [154]:
X_int = X[['type_airbnb_label', 'province_label']]

In [155]:
X_ = X.drop(['type_airbnb_label', 'province_label'],axis=1)

In [156]:
X_=X_.astype(float)
X_.dtypes

guests                     float64
Numberofbeds               float64
Numberofbedrooms           float64
Numberofbathrooms          float64
superhost                  float64
Host_IdentityVerified      float64
waterfront                 float64
compactor                  float64
securitycameras            float64
sauna                      float64
parking                    float64
washingmachine             float64
gameconsole                float64
gym                        float64
balcony                    float64
piano                      float64
swimmingpool               float64
indoorgames                float64
wifi                       float64
hairdryer                  float64
hangers                    float64
bathtub                    float64
hotwater                   float64
iron                       float64
karaoke                    float64
safe                       float64
keypad                     float64
kitchen                    float64
kettle              

In [157]:
X_ = pd.concat([X_,X_int],axis=1)

In [158]:
X_.head(1)

,guests,Numberofbeds,Numberofbedrooms,Numberofbathrooms,superhost,Host_IdentityVerified,waterfront,compactor,securitycameras,sauna,parking,washingmachine,gameconsole,gym,balcony,piano,swimmingpool,indoorgames,wifi,hairdryer,hangers,bathtub,hotwater,iron,karaoke,safe,keypad,kitchen,kettle,tv,shampooconditioner,essentials,refrigerator,streamingservice,airconditioning,soundsystem,oven,grill,babyfriendly,utensils,linen,bidet,breakfast,staff,alarm,fan,heating,childfriendly,clothesstorage,coffeemaker,dedicatedworkspace,diningtable,dryer,cookingbasics,elevator,beddings,garden,fireextinguisher,firepit,fireplaceguards,firstadkit,enhancedclean,cancellationpolicy,experiencedhost,freeparkingonpremises,greatcheckinexperience,greatcommunication,greatlocation,highlyratedhost,houserules,laptopfriendlyworkspace,outstandinghospitality,petsallowed,selfcheckin,parklingclean,numnearbylistings,numneartouristspots5km,numnearbymall5km,numnearbyrestaurants5km,numnearpublictranspo,distance_airport,num_reviews,within3km,coast750m,type_airbnb_label,province_label
0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,2.0,0.0,39.603528,1.0,0.0,0.0,7,82


In [159]:
X_.dtypes

guests                     float64
Numberofbeds               float64
Numberofbedrooms           float64
Numberofbathrooms          float64
superhost                  float64
Host_IdentityVerified      float64
waterfront                 float64
compactor                  float64
securitycameras            float64
sauna                      float64
parking                    float64
washingmachine             float64
gameconsole                float64
gym                        float64
balcony                    float64
piano                      float64
swimmingpool               float64
indoorgames                float64
wifi                       float64
hairdryer                  float64
hangers                    float64
bathtub                    float64
hotwater                   float64
iron                       float64
karaoke                    float64
safe                       float64
keypad                     float64
kitchen                    float64
kettle              

In [160]:
X_.shape

(11409, 86)

In [34]:
cb.CatBoostRegressor?

In [161]:
train_X, test_X, train_y, test_y = train_test_split(X_, Y,test_size = 0.2, random_state = 100)

In [162]:
train_dataset = cb.Pool(train_X, train_y) 
test_dataset = cb.Pool(test_X, test_y)

In [163]:
# Instantiation
#early_stopping_rounds=10
cbc = cb.CatBoostRegressor(random_seed=42,loss_function='RMSE', verbose=True, cat_features=np.where(X_.dtypes != np.float)[0])

In [164]:
#create the grid

grid = {'max_depth': [5,10],
        'n_estimators': [100,200],
        'learning_rate' : [0.1,0.3],
        'metric_period':[50,60],#calculate metrics once per 50 iterations
        'od_wait':[20,30] #most recent best iteration to wait before stopping
       
       }

In [68]:
GridSearchCV?

In [165]:
#Instantiate GridSearchCV
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring ='r2', cv = 5 )

In [166]:
#Fit the model
gscv.fit(train_X, train_y)

0:	learn: 2927.7111778	total: 19.9ms	remaining: 1.97s
50:	learn: 1965.0409147	total: 1.27s	remaining: 1.22s
99:	learn: 1806.3015452	total: 2.39s	remaining: 0us
0:	learn: 2931.1445463	total: 19.5ms	remaining: 1.93s
50:	learn: 1927.0141089	total: 1.22s	remaining: 1.17s
99:	learn: 1722.0650085	total: 2.31s	remaining: 0us
0:	learn: 2946.3250319	total: 18.5ms	remaining: 1.83s
50:	learn: 1948.2067261	total: 1.18s	remaining: 1.13s
99:	learn: 1772.1627553	total: 2.34s	remaining: 0us
0:	learn: 2886.2755361	total: 20.1ms	remaining: 1.99s
50:	learn: 1902.0398388	total: 1.16s	remaining: 1.11s
99:	learn: 1739.3933145	total: 2.24s	remaining: 0us
0:	learn: 2952.0209271	total: 20.6ms	remaining: 2.04s
50:	learn: 1955.2638446	total: 1.12s	remaining: 1.08s
99:	learn: 1780.3577668	total: 2.21s	remaining: 0us
0:	learn: 2927.7111778	total: 18.9ms	remaining: 1.87s
50:	learn: 1965.0409147	total: 1.28s	remaining: 1.23s
99:	learn: 1806.3015452	total: 2.37s	remaining: 0us
0:	learn: 2931.1445463	total: 19.8ms	rem

180:	learn: 1582.1351108	total: 5.32s	remaining: 559ms
199:	learn: 1544.9549825	total: 5.88s	remaining: 0us
0:	learn: 2954.9522545	total: 32ms	remaining: 6.37s
60:	learn: 1892.3313196	total: 1.75s	remaining: 4s
120:	learn: 1745.0686573	total: 3.74s	remaining: 2.44s
180:	learn: 1610.5458401	total: 5.43s	remaining: 570ms
199:	learn: 1574.3759356	total: 5.99s	remaining: 0us
0:	learn: 2931.9229039	total: 114ms	remaining: 11.3s
50:	learn: 1446.4335594	total: 5.62s	remaining: 5.4s
99:	learn: 1191.5864118	total: 10.9s	remaining: 0us
0:	learn: 2921.3536015	total: 106ms	remaining: 10.5s
50:	learn: 1414.2588311	total: 6.23s	remaining: 5.99s
99:	learn: 1135.5318290	total: 11.7s	remaining: 0us
0:	learn: 2937.6766271	total: 117ms	remaining: 11.6s
50:	learn: 1444.6645188	total: 6.22s	remaining: 5.98s
99:	learn: 1165.3768761	total: 12.3s	remaining: 0us
0:	learn: 2886.8392989	total: 102ms	remaining: 10.1s
50:	learn: 1396.3486321	total: 5.45s	remaining: 5.23s
99:	learn: 1112.1935494	total: 11.1s	remain

180:	learn: 907.3309268	total: 31.3s	remaining: 3.28s
199:	learn: 854.1479661	total: 34.4s	remaining: 0us
0:	learn: 2883.8099731	total: 170ms	remaining: 33.9s
60:	learn: 1345.6601248	total: 10.1s	remaining: 23.1s
120:	learn: 1031.9070264	total: 19.8s	remaining: 12.9s
180:	learn: 856.7805618	total: 29.1s	remaining: 3.05s
199:	learn: 807.6465946	total: 32.1s	remaining: 0us
0:	learn: 2961.2025994	total: 162ms	remaining: 32.2s
60:	learn: 1282.1767613	total: 9.43s	remaining: 21.5s
120:	learn: 1031.8118153	total: 18.2s	remaining: 11.9s
180:	learn: 878.8255950	total: 27.6s	remaining: 2.9s
199:	learn: 812.8701133	total: 31.2s	remaining: 0us
0:	learn: 2743.6118961	total: 34.8ms	remaining: 3.45s
50:	learn: 1717.5689655	total: 2.36s	remaining: 2.27s
99:	learn: 1473.1963682	total: 4.75s	remaining: 0us
0:	learn: 2755.4219053	total: 35.7ms	remaining: 3.53s
50:	learn: 1633.3891697	total: 2.09s	remaining: 2.01s
99:	learn: 1402.5365016	total: 4.05s	remaining: 0us
0:	learn: 2776.8044436	total: 46.2ms	re

60:	learn: 1574.7194470	total: 2.34s	remaining: 5.33s
120:	learn: 1353.7489944	total: 4.68s	remaining: 3.06s
180:	learn: 1194.1501670	total: 6.96s	remaining: 730ms
199:	learn: 1150.8354097	total: 7.68s	remaining: 0us
0:	learn: 2752.9916821	total: 33ms	remaining: 6.56s
60:	learn: 1617.3758230	total: 2.33s	remaining: 5.32s
120:	learn: 1367.0691920	total: 4.61s	remaining: 3.01s
180:	learn: 1203.9773086	total: 6.98s	remaining: 733ms
199:	learn: 1166.6402702	total: 7.76s	remaining: 0us
0:	learn: 2700.8203371	total: 36ms	remaining: 7.16s
60:	learn: 1583.8011981	total: 2.47s	remaining: 5.63s
120:	learn: 1361.1480898	total: 4.78s	remaining: 3.12s
180:	learn: 1212.6809968	total: 7.13s	remaining: 748ms
199:	learn: 1178.3506015	total: 7.86s	remaining: 0us
0:	learn: 2770.5213523	total: 31.9ms	remaining: 6.34s
60:	learn: 1646.2075519	total: 2.27s	remaining: 5.18s
120:	learn: 1395.0618815	total: 4.69s	remaining: 3.06s
180:	learn: 1227.7028809	total: 7.04s	remaining: 739ms
199:	learn: 1191.4010612	to

120:	learn: 541.8830480	total: 15.6s	remaining: 10.2s
180:	learn: 353.3178109	total: 23.6s	remaining: 2.47s
199:	learn: 311.1735756	total: 26.1s	remaining: 0us
0:	learn: 2731.8921799	total: 136ms	remaining: 27s
60:	learn: 830.6232576	total: 8.08s	remaining: 18.4s
120:	learn: 497.6037320	total: 16.3s	remaining: 10.6s
180:	learn: 326.9820052	total: 24.6s	remaining: 2.58s
199:	learn: 295.7962135	total: 27.1s	remaining: 0us
0:	learn: 2745.5141496	total: 129ms	remaining: 25.7s
60:	learn: 928.1926799	total: 7.86s	remaining: 17.9s
120:	learn: 535.8253100	total: 15.5s	remaining: 10.1s
180:	learn: 347.4518897	total: 23.3s	remaining: 2.44s
199:	learn: 307.2186051	total: 25.7s	remaining: 0us
0:	learn: 2691.8375333	total: 130ms	remaining: 25.8s
60:	learn: 827.8109762	total: 7.65s	remaining: 17.4s
120:	learn: 475.2222424	total: 15.2s	remaining: 9.91s
180:	learn: 303.0817846	total: 22.1s	remaining: 2.32s
199:	learn: 270.5996010	total: 24.3s	remaining: 0us
0:	learn: 2778.0684137	total: 129ms	remainin

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001E50DDAF430>,
             param_grid={'learning_rate': [0.1, 0.3], 'max_depth': [5, 10],
                         'metric_period': [50, 60], 'n_estimators': [100, 200],
                         'od_wait': [20, 30]},
             scoring='r2')

In [167]:
#returns the estimator with the best performance
print(gscv.best_estimator_)

In [168]:
#returns the best score
print(gscv.best_score_)

0.5576438853158948


In [169]:
#returns the best parameters
print(gscv.best_params_)


{'learning_rate': 0.1, 'max_depth': 5, 'metric_period': 50, 'n_estimators': 200, 'od_wait': 20}


In [133]:
#Instantiate GridSearchCV without cv
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring ='r2')

In [134]:
gscv.fit(train_X, train_y)

0:	learn: 2927.7111778	total: 22.6ms	remaining: 2.24s
50:	learn: 1965.0409147	total: 1.16s	remaining: 1.12s
99:	learn: 1806.3015452	total: 2.19s	remaining: 0us
0:	learn: 2931.1445463	total: 19.7ms	remaining: 1.95s
50:	learn: 1927.0141089	total: 1.21s	remaining: 1.17s
99:	learn: 1722.0650085	total: 2.54s	remaining: 0us
0:	learn: 2946.3250319	total: 18.3ms	remaining: 1.81s
50:	learn: 1948.2067261	total: 1.39s	remaining: 1.34s
99:	learn: 1772.1627553	total: 2.68s	remaining: 0us
0:	learn: 2886.2755361	total: 20.9ms	remaining: 2.07s
50:	learn: 1902.0398388	total: 1.17s	remaining: 1.13s
99:	learn: 1739.3933145	total: 2.28s	remaining: 0us
0:	learn: 2952.0209271	total: 25.7ms	remaining: 2.54s
50:	learn: 1955.2638446	total: 1.28s	remaining: 1.23s
99:	learn: 1780.3577668	total: 2.57s	remaining: 0us
0:	learn: 2927.7111778	total: 29.5ms	remaining: 2.92s
50:	learn: 1965.0409147	total: 1.6s	remaining: 1.54s
99:	learn: 1806.3015452	total: 2.71s	remaining: 0us
0:	learn: 2931.1445463	total: 18.7ms	rema

180:	learn: 1582.1351108	total: 4.13s	remaining: 433ms
199:	learn: 1544.9549825	total: 4.56s	remaining: 0us
0:	learn: 2954.9522545	total: 19.1ms	remaining: 3.81s
60:	learn: 1892.3313196	total: 1.38s	remaining: 3.14s
120:	learn: 1745.0686573	total: 2.89s	remaining: 1.88s
180:	learn: 1610.5458401	total: 4.32s	remaining: 454ms
199:	learn: 1574.3759356	total: 4.77s	remaining: 0us
0:	learn: 2931.9229039	total: 98.5ms	remaining: 9.75s
50:	learn: 1446.4335594	total: 5.42s	remaining: 5.2s
99:	learn: 1191.5864118	total: 10.1s	remaining: 0us
0:	learn: 2921.3536015	total: 111ms	remaining: 11s
50:	learn: 1414.2588311	total: 5.58s	remaining: 5.36s
99:	learn: 1135.5318290	total: 10.6s	remaining: 0us
0:	learn: 2937.6766271	total: 100ms	remaining: 9.9s
50:	learn: 1444.6645188	total: 5.33s	remaining: 5.12s
99:	learn: 1165.3768761	total: 10.9s	remaining: 0us
0:	learn: 2886.8392989	total: 106ms	remaining: 10.5s
50:	learn: 1396.3486321	total: 5.39s	remaining: 5.18s
99:	learn: 1112.1935494	total: 11s	remai

180:	learn: 907.3309268	total: 21.2s	remaining: 2.23s
199:	learn: 854.1479661	total: 23.7s	remaining: 0us
0:	learn: 2883.8099731	total: 134ms	remaining: 26.6s
60:	learn: 1345.6601248	total: 7.63s	remaining: 17.4s
120:	learn: 1031.9070264	total: 15.5s	remaining: 10.1s
180:	learn: 856.7805618	total: 22.9s	remaining: 2.4s
199:	learn: 807.6465946	total: 25.1s	remaining: 0us
0:	learn: 2961.2025994	total: 126ms	remaining: 25.1s
60:	learn: 1282.1767613	total: 7.71s	remaining: 17.6s
120:	learn: 1031.8118153	total: 14.2s	remaining: 9.26s
180:	learn: 878.8255950	total: 21.2s	remaining: 2.22s
199:	learn: 812.8701133	total: 23.7s	remaining: 0us
0:	learn: 2743.6118961	total: 21.1ms	remaining: 2.09s
50:	learn: 1717.5689655	total: 1.26s	remaining: 1.21s
99:	learn: 1473.1963682	total: 2.55s	remaining: 0us
0:	learn: 2755.4219053	total: 23.9ms	remaining: 2.37s
50:	learn: 1633.3891697	total: 1.62s	remaining: 1.55s
99:	learn: 1402.5365016	total: 3.01s	remaining: 0us
0:	learn: 2776.8044436	total: 25.9ms	re

60:	learn: 1574.7194470	total: 1.46s	remaining: 3.33s
120:	learn: 1353.7489944	total: 2.93s	remaining: 1.91s
180:	learn: 1194.1501670	total: 4.38s	remaining: 460ms
199:	learn: 1150.8354097	total: 4.84s	remaining: 0us
0:	learn: 2752.9916821	total: 22.7ms	remaining: 4.51s
60:	learn: 1617.3758230	total: 1.46s	remaining: 3.32s
120:	learn: 1367.0691920	total: 3.23s	remaining: 2.11s
180:	learn: 1203.9773086	total: 4.68s	remaining: 491ms
199:	learn: 1166.6402702	total: 5.12s	remaining: 0us
0:	learn: 2700.8203371	total: 22.5ms	remaining: 4.47s
60:	learn: 1583.8011981	total: 1.43s	remaining: 3.25s
120:	learn: 1361.1480898	total: 2.85s	remaining: 1.86s
180:	learn: 1212.6809968	total: 4.27s	remaining: 448ms
199:	learn: 1178.3506015	total: 4.74s	remaining: 0us
0:	learn: 2770.5213523	total: 22.8ms	remaining: 4.53s
60:	learn: 1646.2075519	total: 1.45s	remaining: 3.29s
120:	learn: 1395.0618815	total: 2.86s	remaining: 1.87s
180:	learn: 1227.7028809	total: 4.28s	remaining: 449ms
199:	learn: 1191.401061

120:	learn: 541.8830480	total: 16.4s	remaining: 10.7s
180:	learn: 353.3178109	total: 26.2s	remaining: 2.75s
199:	learn: 311.1735756	total: 29.3s	remaining: 0us
0:	learn: 2731.8921799	total: 168ms	remaining: 33.4s
60:	learn: 830.6232576	total: 10.8s	remaining: 24.6s
120:	learn: 497.6037320	total: 20s	remaining: 13.1s
180:	learn: 326.9820052	total: 27.3s	remaining: 2.87s
199:	learn: 295.7962135	total: 29.7s	remaining: 0us
0:	learn: 2745.5141496	total: 118ms	remaining: 23.6s
60:	learn: 928.1926799	total: 7.34s	remaining: 16.7s
120:	learn: 535.8253100	total: 14.3s	remaining: 9.34s
180:	learn: 347.4518897	total: 21.6s	remaining: 2.26s
199:	learn: 307.2186051	total: 23.8s	remaining: 0us
0:	learn: 2691.8375333	total: 121ms	remaining: 24.1s
60:	learn: 827.8109762	total: 6.87s	remaining: 15.7s
120:	learn: 475.2222424	total: 13.9s	remaining: 9.07s
180:	learn: 303.0817846	total: 20.8s	remaining: 2.18s
199:	learn: 270.5996010	total: 22.9s	remaining: 0us
0:	learn: 2778.0684137	total: 109ms	remainin

GridSearchCV(estimator=<catboost.core.CatBoostRegressor object at 0x000001E50DD9F220>,
             param_grid={'learning_rate': [0.1, 0.3], 'max_depth': [5, 10],
                         'metric_period': [50, 60], 'n_estimators': [100, 200],
                         'od_wait': [20, 30]},
             scoring='r2')

In [136]:
pred = gscv.predict(test_X)
rmse = (np.sqrt(MSE(test_y, pred)))
mse = (MSE(test_y, pred))
r2 = r2_score(test_y, pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('MSE: {:.2f}'.format(mse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 1951.03
MSE: 3806509.97
R2: 0.59


In [140]:
rmse = (np.sqrt(MSE(train_y, pred)))
mse = (MSE(train_y, pred))
r2 = r2_score(train_y, pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('MSE: {:.2f}'.format(mse))
print('R2: {:.2f}'.format(r2))

ValueError: Found input variables with inconsistent numbers of samples: [9127, 2282]

In [54]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])